Модуль №2 з дісципліни "Штучні нейронні мережі та машинне навчання" Рогачової Вікторії


Виконуємо імпорт необхідних бібліотек та завантажити датасет у датафрейм Pandas.

In [75]:
import numpy as np
import pandas as pd
import os

from keras.layers import Dense
from keras.models import Sequential

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.pipeline import Pipeline

In [89]:
from tensorflow import keras
from tensorflow.keras import layers

In [12]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [82]:
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/bank.csv')
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes


Перевіряємо на наявність missing data, якщо є характеристики, що містять missing data, заповнити їх за допомогою класу SimpleImputer за стратегією “mean” для числових характеристик та стратегії “most frecquent” для категоріальних.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11162 entries, 0 to 11161
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        11162 non-null  int64 
 1   job        11162 non-null  object
 2   marital    11162 non-null  object
 3   education  11162 non-null  object
 4   default    11162 non-null  object
 5   balance    11162 non-null  int64 
 6   housing    11162 non-null  object
 7   loan       11162 non-null  object
 8   contact    11162 non-null  object
 9   day        11162 non-null  int64 
 10  month      11162 non-null  object
 11  duration   11162 non-null  int64 
 12  campaign   11162 non-null  int64 
 13  pdays      11162 non-null  int64 
 14  previous   11162 non-null  int64 
 15  poutcome   11162 non-null  object
 16  deposit    11162 non-null  object
dtypes: int64(7), object(10)
memory usage: 1.4+ MB


In [11]:
df.shape

(11162, 17)

In [6]:
imp_mean = SimpleImputer(missing_values=np.nan,strategy='mean')
imp_freq = SimpleImputer(missing_values=np.nan,strategy='most_frequent')

За допомогою методів Scikit-Learn (LabelEncoder, OneHotEncoder) перетворити категоріальні мітки (окремо значення незалежних змінних та окремо для значень залежної змінної) у датасеті (bank.csv) у числові коди, а їх, у свою чергу, у бінарні вектори.

In [57]:
df_cols = df.columns[1:5]
df_cols=df_cols.append(df.columns[6:9])
df_cols

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact'], dtype='object')

In [58]:
df_X = df[df_cols]
df_X.head()

,job,marital,education,default,housing,loan,contact
0,admin.,married,secondary,no,yes,no,unknown
1,admin.,married,secondary,no,no,no,unknown
2,technician,married,secondary,no,yes,no,unknown
3,services,married,secondary,no,yes,no,unknown
4,admin.,married,tertiary,no,no,no,unknown


In [60]:
y = df['deposit']
y.head()

0    yes
1    yes
2    yes
3    yes
4    yes
Name: deposit, dtype: object

In [61]:
f1 = lambda x:1 if x == 'yes' else 0
y1 = y.map(f1)
y1.head()

0    1
1    1
2    1
3    1
4    1
Name: deposit, dtype: int64

In [63]:
onehot_encoder = OneHotEncoder(sparse=False)

In [66]:
df_y_encoded = y1.copy()
arr1 = np.array(df_y_encoded)
arr1 = arr1.reshape(len(arr1),1)
y_onehot_encoded = onehot_encoder.fit_transform(arr1)
df_y_encoded = onehot_encoded
df_y_encoded

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]])

In [62]:
le = LabelEncoder()
df_X1 = pd.DataFrame()
for i in df_X.columns:
  df_X1[i]=le.fit_transform(df_X[i])
df_X1.head()

,job,marital,education,default,housing,loan,contact
0,0,1,1,0,1,0,2
1,0,1,1,0,0,0,2
2,9,1,1,0,1,0,2
3,7,1,1,0,1,0,2
4,0,1,2,0,0,0,2


In [64]:
df_X_encoded = df_X.copy()
for i in range(df_cols.size):
  arr = np.array(df_X1[df_cols[i]])
  arr = arr.reshape(len(arr),1)
  onehot_encoded = onehot_encoder.fit_transform(arr)
  df_X_encoded[df_cols[i]] = onehot_encoded
df_X_encoded.head()

,job,marital,education,default,housing,loan,contact
0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
1,1.0,0.0,0.0,1.0,1.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,1.0,0.0,0.0,1.0,1.0,1.0,0.0


Створюємо функцію, що повертає модель нейронної мережі з двома прихованими
шарами (передбачити можливість використовувати параметри цієї функції для зміни
типу оптимізатора та схеми ініціалізації ваг нейромережі)

In [92]:
def create_model(optimizer = 'rmsprop', init='glorot_uniform'):
  model= keras.Sequential()
  model.add(layers.Dense(12,input_dim=7,kernel_initializer='uniform',activation='relu'))
  model.add(layers.Dense(8,kernel_initializer='uniform',activation='relu'))
  model.add(layers.Dense(1,kernel_initializer='uniform',activation='sigmoid'))
  
  model.compile(  optimizer = 'adam', loss = 'BinaryCrossentropy', metrics = ['accuracy'])
  return model

Створюємо модель, використовуючи екземпляр класу KerasClassifier

In [93]:
model1 = KerasClassifier(build_fn=create_model,verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


Використовуючи пошук сітки з крос-валідацією проводимо дослідження отриманої моделі

In [94]:
optimizers = ['rmsprop','adam']
init = ['glorot_uniform','normal','uniform']
epochs = np.array([50,100,150])
batches = np.array([5,10,20])
param_grid = dict(optimizer=optimizers,nb_epoch=epochs,batch_size=batches,init=init)
grid = GridSearchCV(estimator=model1, param_grid=param_grid)

In [96]:
kfold = StratifiedKFold(n_splits = 20, shuffle = True, random_state=27)
results = cross_val_score(model1, df_X1, y1, cv=kfold)
print('Baseline: %.2f%% (%.2f%%)' % (results.mean()*100,results.std()*100))

Baseline: 63.09% (1.43%)


In [97]:
grid_result = grid.fit(df_X1,y1)

In [98]:
print('Best: %f using %s' % (grid_result.best_score_,grid_result.best_params_))

Best: 0.384645 using {'batch_size': 5, 'init': 'normal', 'nb_epoch': 150, 'optimizer': 'adam'}


На основі отриманих у попередньому пункті результатів сформували датафрейм та виводимо перші 5 його рядків.

In [99]:
res = pd.DataFrame(grid_result.cv_results_)
res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_batch_size,param_init,param_nb_epoch,param_optimizer,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.856662,0.257817,0.604525,0.086857,5,glorot_uniform,50,rmsprop,"{'batch_size': 5, 'init': 'glorot_uniform', 'n...",0.341693,0.0,0.598118,0.537634,0.323925,0.360274,0.209519,11
1,2.863178,0.225052,0.602195,0.090433,5,glorot_uniform,50,adam,"{'batch_size': 5, 'init': 'glorot_uniform', 'n...",0.345275,0.0,0.629032,0.438172,0.347222,0.351940,0.204032,16
2,2.846744,0.197544,0.689225,0.108519,5,glorot_uniform,100,rmsprop,"{'batch_size': 5, 'init': 'glorot_uniform', 'n...",0.317510,0.0,0.645609,0.360215,0.449373,0.354541,0.210151,14
3,2.850584,0.214044,0.691027,0.102221,5,glorot_uniform,100,adam,"{'batch_size': 5, 'init': 'glorot_uniform', 'n...",0.343932,0.0,0.597222,0.437276,0.375448,0.350776,0.195917,19
4,2.944758,0.300334,0.605298,0.083113,5,glorot_uniform,150,rmsprop,"{'batch_size': 5, 'init': 'glorot_uniform', 'n...",0.344828,0.0,0.616487,0.435484,0.345878,0.348535,0.200428,24
